# "2019 June One Day Holding Long Short Portfolio BackTest"
> "2019 June Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [105]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [106]:
#hide_input
#df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
#df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')

In [107]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190601].loc[df_tes.Date<20190701].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [108]:
#hide
#df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_agg['cumreturn1']=0.0
#for l in range(len(df_agg)):
#    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
#df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_ag['cumreturn1a']=0.0
#for l in range(len(df_ag)):
#    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [109]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [110]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-06-03,-1.0,5.220027,0.947800,1.021432,1.020645
1,2019-06-04,-1.0,2.651797,0.922666,1.029769,1.029001
2,2019-06-05,-1.0,-0.878273,0.930770,1.036087,1.036297
3,2019-06-05,1.0,-0.737572,0.923904,1.036087,1.036297
4,2019-06-06,-1.0,8.504380,0.845332,1.046964,1.046905
5,2019-06-07,-1.0,8.829297,0.770695,1.051843,1.050077
6,2019-06-07,1.0,1.429822,0.781715,1.051843,1.050077
7,2019-06-10,-1.0,1.336956,0.771264,1.051475,1.049506
8,2019-06-11,-1.0,-6.705440,0.822980,1.049332,1.047746
9,2019-06-11,1.0,0.769231,0.829311,1.049332,1.047746


In [111]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [112]:
#hide
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [113]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [114]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-06-05,1.0,-0.737572,0.992624,1.006136,1.007091
1,2019-06-07,1.0,1.429822,1.006817,1.021436,1.020482
2,2019-06-11,1.0,0.769231,1.014562,1.018998,1.018217
3,2019-06-12,1.0,-1.255522,1.001824,1.023173,1.022209
4,2019-06-18,1.0,12.195122,1.123997,1.035494,1.037762
5,2019-06-27,1.0,1.362862,1.139316,1.034949,1.038646


In [115]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [116]:
#hide
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [117]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [118]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-06-03,-1.0,5.220027,0.947800,1.021432,1.020645
1,2019-06-04,-1.0,2.651797,0.922666,1.029769,1.029001
2,2019-06-05,-1.0,-0.878273,0.930770,1.036087,1.036297
3,2019-06-06,-1.0,8.504380,0.851613,1.046964,1.046905
4,2019-06-07,-1.0,8.829297,0.776422,1.051843,1.050077
5,2019-06-10,-1.0,1.336956,0.766041,1.051475,1.049506
6,2019-06-11,-1.0,-6.705440,0.817408,1.049332,1.047746
7,2019-06-12,-1.0,6.682132,0.762788,1.053632,1.051853
8,2019-06-13,-1.0,-18.208358,0.901679,1.051934,1.051162
9,2019-06-14,-1.0,-5.224050,0.948783,1.052914,1.052085


In [119]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)